In [8]:
pip install googletrans

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 7.3 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.0.0-py3-none-any.whl size=15715 sha256=e7967b57575803f103eb3bcba3a87c7f194b29cfe9ad518a33e38c64bef8e311
  Stored in directory: /root/.cache/pip/wheels/b3/81/ea/8b030407f8ebfc2f857814e086bb22ca2d4fea1a7be63652ab
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling chard

In [2]:
import pandas as pd
df = pd.read_csv('/content/cyberbullying_tweets.csv')

In [3]:
df

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying
...,...,...
47687,"Black ppl aren't expected to do anything, depe...",ethnicity
47688,Turner did not withhold his disappointment. Tu...,ethnicity
47689,I swear to God. This dumb nigger bitch. I have...,ethnicity
47690,Yea fuck you RT @therealexel: IF YOURE A NIGGE...,ethnicity


In [5]:
import re

def remove_extra_spaces(text):
    text = text.strip()
    text = re.sub(r'\s+', ' ', text)
    return text

def remove_words_with_at(text):
    text_without_at = re.sub(r'@\w+\b', '', text)
    text_cleaned = re.sub(r'\s+', ' ', text_without_at).strip()
    return text_cleaned

In [6]:
# Removemos las citas de usuarios ya que estas palabras no son necesarias para determinar odio
df['tweet_text'] = df['tweet_text'].apply(remove_words_with_at)

In [9]:
from googletrans import Translator
from multiprocessing import Pool
import pandas as pd

def back_translate(text):
    translator = Translator()
    try:
        texto_espanol = translator.translate(text, dest='es').text
        texto_ingles = translator.translate(texto_espanol, dest='en').text
        return texto_ingles
    except Exception as e:
        return text

def apply_multiprocessing(data, func):
    with Pool() as pool:
        result = pool.map(func, data)
    return result


# Selecciono aleatoriamente el 10% de los registros de las clases 'other_cyberbullying' y 'age'
df_other_cb_sample = df[df['cyberbullying_type'] == 'other_cyberbullying'].sample(frac=0.1, random_state=42)
df_age_sample = df[df['cyberbullying_type'] == 'age'].sample(frac=0.1, random_state=42)

# Aplico la traducción de ida y vuelta a las muestras obtenidas
df_other_cb_sample['tweet_text'] = apply_multiprocessing(df_other_cb_sample['tweet_text'], back_translate)
df_age_sample['tweet_text'] = apply_multiprocessing(df_age_sample['tweet_text'], back_translate)

# Aumento el DataFrame original con los registros traducidos de ida y vuelta
df_augmented = pd.concat([df, df_other_cb_sample, df_age_sample])
df_augmented.reset_index(drop=True, inplace=True)


In [10]:
df_augmented.to_csv('dataset_augmented_crosslingual.csv', index=False)